This part is a repeated step of part 4 using logistic regression. This part was made by mistake as logistic regression had higher test score, but I found out gradient boosting had better test score after correction in the coding.

In [64]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

df = pd.read_csv('data/train.csv')

df.drop(columns = ["Name", "PassengerId", "Cabin", "Ticket"], inplace=True)

df.Age.fillna(df.Age.mean(), inplace=True)
df.Embarked.fillna('N/A', inplace=True)

X = df.drop(columns = ["Survived"])
y = df.Survived

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state=1)

from sklearn.linear_model import LogisticRegression

from imblearn.pipeline import Pipeline

import data_preparation as dp

In [66]:
X_train_labeled = X_train.copy()
X_test_labeled = X_test.copy()

col = ["Sex", "Embarked"]
for c in col:
    X_train_labeled[c] = le.fit_transform(X_train[c].astype('str'))
    X_test_labeled[c] = le.transform(X_test[c].astype('str'))
    
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_labeled, y_train) 

lr = LogisticRegression(random_state=1)

In [67]:
pipe = Pipeline(steps=[('smote', smote),
                       ('lr', LogisticRegression(random_state=1))])

pipe.fit(X_train_labeled, y_train)

Pipeline(steps=[('smote', SMOTE(random_state=1)),
                ('lr', LogisticRegression(random_state=1))])

In [68]:
dp.scores(X_train_labeled, y_train, X_test_labeled, y_test, pipe)

Pipeline(steps=[('smote', SMOTE(random_state=1)),
                ('lr', LogisticRegression(random_state=1))])

CV score:     81.24%
X-test score: 79.82%
RMSE:         0.4492

Train score
              precision    recall  f1-score   support

           0       0.84      0.81      0.83       421
           1       0.70      0.74      0.72       247

    accuracy                           0.79       668
   macro avg       0.77      0.78      0.77       668
weighted avg       0.79      0.79      0.79       668



X-test score

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       128
           1       0.77      0.75      0.76        95

    accuracy                           0.80       223
   macro avg       0.79      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223





In [69]:
from sklearn.model_selection import GridSearchCV

# instantiate grid search for hyperparameter tuning
param = {'lr__max_iter': [100, 200, 300],
         'lr__solver':  ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
        }
gs = GridSearchCV(estimator=pipe,
                  param_grid=param,
                  cv=3)

gs.fit(X_train_labeled, y_train)
gs.best_params_

{'lr__max_iter': 100, 'lr__solver': 'newton-cg'}

In [70]:
pipe_tuned = Pipeline(steps=[('smote', smote),
                             ('lr', LogisticRegression(random_state=1, solver = 'newton-cg'))])

pipe_tuned.fit(X_train_labeled, y_train)
dp.scores(X_train_labeled, y_train, X_test_labeled, y_test, pipe_tuned)

Pipeline(steps=[('smote', SMOTE(random_state=1)),
                ('lr', LogisticRegression(random_state=1, solver='newton-cg'))])

CV score:     81.04%
X-test score: 79.82%
RMSE:         0.4492

Train score
              precision    recall  f1-score   support

           0       0.84      0.81      0.83       421
           1       0.70      0.74      0.72       247

    accuracy                           0.79       668
   macro avg       0.77      0.78      0.77       668
weighted avg       0.79      0.79      0.79       668



X-test score

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       128
           1       0.77      0.75      0.76        95

    accuracy                           0.80       223
   macro avg       0.79      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223





In [74]:
pipe.steps

[('smote', SMOTE(random_state=1)),
 ('lr', LogisticRegression(random_state=1, solver='newton-cg'))]

In [79]:
pipe.steps[1][1].coef_

array([[-1.14105186e+00, -2.92995323e+00, -5.00089387e-02,
        -5.22121987e-01, -4.89563975e-02,  7.09411601e-04,
        -1.98396046e-01]])

In [76]:
X_train_resampled, y_train_resampled = pipe.steps[0][1].fit_sample(X_train_labeled, y_train) 

In [89]:
feature_imp = np.std(X_train_resampled, 0)

In [119]:
feature_imp.index

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [141]:
feat_impt = pd.DataFrame({'std': np.std(X_train_resampled, 0).values,
                          'coef': pipe_tuned.steps[1][1].coef_[0]},
                         index=feature_imp.index)

In [165]:
feat_impt["feat_imp"] = np.absolute(feat_impt.iloc[:,0] * feat_impt.iloc[:,1])

In [167]:
feat_impt.sort_values(by='feat_imp', ascending=False)

,std,coef,feat_imp
Sex,0.498169,-2.929953,1.459611
Pclass,0.847493,-1.141052,0.967033
Age,13.666580,-0.050009,0.683451
SibSp,0.939060,-0.522122,0.490304
Embarked,1.162025,-0.198396,0.230541
Parch,0.750605,-0.048956,0.036747
Fare,48.139221,0.000709,0.034151
